In [10]:
import json
import boto3
import botocore
from IPython.display import display, Markdown
import time

In [11]:
# Initialize Bedrock client
session = boto3.session.Session()
region = session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)

In [17]:
# Define model IDs that will be used in this module
MODELS = {
    "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    "Claude 3.5 Sonnet": "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    "Claude 3.5 Haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
    "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
    "DeepSeek-R1": "us.deepseek.r1-v1:0",
    "Meta Llama 3.1 70B Instruct": "us.meta.llama3-1-70b-instruct-v1:0"
}

In [18]:
# Utility function to display model responses in a more readable format
def display_response(response, model_name=None):
    if model_name:
        display(Markdown(f"### Response from {model_name}"))
    display(Markdown(response))
    print("\n" + "-"*80 + "\n")

In [19]:
text_to_summarize = """
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we're also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock's serverless experience, \
customers can easily find the right model for what they're trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

In [20]:
# Create prompt for summarization
prompt = f"""Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.
<text>
{text_to_summarize}
</text>
"""

In [21]:
# Create request body for Claude 3.7 Sonnet
claude_body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.9,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }
    ],
})

In [22]:
# Send request to Claude 3.7 Sonnet
try:
    response = bedrock.invoke_model(
        modelId=MODELS["Claude 3.7 Sonnet"],
        body=claude_body,
        accept="application/json",
        contentType="application/json"
    )
    response_body = json.loads(response.get('body').read())
    
    # Extract and display the response text
    claude_summary = response_body["content"][0]["text"]
    display_response(claude_summary, "Claude 3.7 Sonnet (Invoke Model API)")
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
            \nTo troubleshoot this issue please refer to the following resources.\
            \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
            \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Invoke Model API)

# Summary

Amazon announced Amazon Bedrock, a new service that provides API access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. Bedrock aims to democratize access to generative AI by offering an easy way for customers to build and scale applications. The service includes text and image models, including Amazon's newly announced Titan LLMs. As a serverless AWS managed service, Bedrock allows customers to find appropriate models, customize them privately with their own data, and integrate them into applications using familiar AWS tools without managing infrastructure. The service integrates with Amazon SageMaker features like Experiments and Pipelines.


--------------------------------------------------------------------------------



In [23]:
# Create a converse request with our summarization task
converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": f"Please provide a concise summary of the following text in 2-3 sentences. Text to summarize: {text_to_summarize}"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.4,
        "topP": 0.9,
        "maxTokens": 500
    }
}

In [24]:
# Call Claude 3.7 Sonnet with Converse API
try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )
    
    # Extract the model's response
    claude_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(claude_converse_response, "Claude 3.7 Sonnet (Converse API)")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Converse API)

AWS has launched Amazon Bedrock, a new service providing API access to foundation models (FMs) from various AI companies, including Amazon's own Titan LLMs. The service democratizes generative AI by offering a serverless experience where customers can easily find, customize, and deploy text and image models without managing infrastructure. Bedrock integrates with existing AWS tools and allows private customization with customers' own data.


--------------------------------------------------------------------------------



In [25]:
# call different models with the same converse request
results = {}    
for model_name, model_id in MODELS.items(): # looping over all models defined above
        try:
            start_time = time.time()
            response = bedrock.converse(
                modelId=model_id,
                messages=converse_request["messages"],
                inferenceConfig=converse_request["inferenceConfig"] if "inferenceConfig" in converse_request else None
            )
            end_time = time.time()
            
            # Extract the model's response using the correct structure
            model_response = response["output"]["message"]["content"][0]["text"]
            response_time = round(end_time - start_time, 2)
            
            results[model_name] = {
                "response": model_response,
                "time": response_time
            }
            
            print(f"✅ Successfully called {model_name} (took {response_time} seconds)")
            
        except Exception as e:
            print(f"❌ Error calling {model_name}: {str(e)}")
            results[model_name] = {
                "response": f"Error: {str(e)}",
                "time": None
            }

✅ Successfully called Claude 3.7 Sonnet (took 3.73 seconds)
✅ Successfully called Claude 3.5 Sonnet (took 3.12 seconds)
✅ Successfully called Claude 3.5 Haiku (took 2.82 seconds)
✅ Successfully called Amazon Nova Pro (took 1.08 seconds)
✅ Successfully called Amazon Nova Micro (took 0.53 seconds)
✅ Successfully called DeepSeek-R1 (took 1.88 seconds)
✅ Successfully called Meta Llama 3.1 70B Instruct (took 4.2 seconds)


In [26]:
# Display results in a formatted way
for model_name, result in results.items():
    if "Error" not in result["response"]:
        display(Markdown(f"### {model_name} (took {result['time']} seconds)"))
        display(Markdown(result["response"]))
        print("-" * 80)

### Claude 3.7 Sonnet (took 3.73 seconds)

AWS has launched Amazon Bedrock, a new service providing API access to foundation models (FMs) from various AI companies, including Amazon's own Titan LLMs. Bedrock democratizes generative AI by offering a serverless experience where customers can easily select, customize, and deploy FMs for text and image applications without managing infrastructure. The service integrates with existing AWS tools and provides a secure, scalable environment for developers to build AI applications.

--------------------------------------------------------------------------------


### Claude 3.5 Sonnet (took 3.12 seconds)

Amazon has announced Bedrock, a new service that provides API access to various foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon itself, including two new large language models from Amazon's Titan series. Bedrock aims to democratize access to generative AI by offering a user-friendly, serverless experience that allows customers to easily find, customize, and deploy FMs for text and image applications. The service integrates with existing AWS tools and eliminates the need for infrastructure management, making it simpler for builders to scale their AI-based applications.

...



AWS has introduced Amazon Bedrock, a service enabling developers to build and scale generative AI applications using foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon—including Amazon’s new Titan models—via a serverless API. Bedrock simplifies access to diverse FMs for text and image tasks, allowing customization with private data, seamless AWS integration, and deployment without infrastructure management. It also supports tools like Amazon SageMaker for testing and scaling models, making AI development more accessible and efficient.

--------------------------------------------------------------------------------


### Meta Llama 3.1 70B Instruct (took 4.2 seconds)



Here is a 3-sentence summary of the text:

Amazon Web Services (AWS) has launched Amazon Bedrock, a new service that provides access to foundation models (FMs) from leading AI labs via an API. Bedrock makes it easy for customers to build and scale generative AI-based applications using FMs, with a serverless experience that allows for private customization and seamless integration with AWS tools. The service offers a range of powerful FMs for text and images, including Amazon's new Titan FMs, and is designed to democratize access to AI technology for all builders.

--------------------------------------------------------------------------------


In [27]:
# Regular model invocation (standard region)
standard_response = bedrock.converse(
    modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",  # Standard model ID
    messages=converse_request["messages"]
)

# Cross-region inference (note the "us." prefix)
cris_response = bedrock.converse(
    modelId="us.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Cross-region model ID with regional prefix
    messages=converse_request["messages"]
)

# Print responses
print("Standard response:", standard_response["output"]["message"]["content"][0]["text"])
print("Cross-region response:", cris_response["output"]["message"]["content"][0]["text"])

Standard response: Amazon has announced Bedrock, a new service providing API access to foundation models (FMs) from various AI companies and Amazon itself. Bedrock aims to democratize access to generative AI by offering a user-friendly platform for building and scaling AI applications. The service includes Amazon's own Titan FMs and integrates with existing AWS tools, allowing customers to easily customize, deploy, and manage AI models without infrastructure concerns.
Cross-region response: Amazon has announced Bedrock, a new service that provides API access to Foundation Models (FMs) from various AI companies, including Amazon's own Titan models. Bedrock aims to democratize access to generative AI by offering a user-friendly, serverless platform for building and scaling AI applications. The service allows customers to easily find, customize, and deploy FMs using familiar AWS tools and infrastructure, without the need to manage complex systems.


In [28]:
# Example of a multi-turn conversation with Converse API
multi_turn_messages = [
    {
        "role": "user",
        "content": [{"text": f"Please summarize this text: {text_to_summarize}"}]
    },
    {
        "role": "assistant",
        "content": [{"text": results["Claude 3.7 Sonnet"]["response"]}]
    },
    {
        "role": "user",
        "content": [{"text": "Can you make this summary even shorter, just 1 sentence?"}]
    }
]

try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=multi_turn_messages,
        inferenceConfig={"temperature": 0.2, "maxTokens": 500}
    )
    
    # Extract the model's response using the correct structure
    follow_up_response = response["output"]["message"]["content"][0]["text"]
    display_response(follow_up_response, "Claude 3.7 Sonnet (Multi-turn conversation)")
    
except Exception as e:
    print(f"Error: {str(e)}")

### Response from Claude 3.7 Sonnet (Multi-turn conversation)

Amazon Bedrock is AWS's new serverless service that provides API access to foundation models from multiple AI companies, allowing developers to easily select, customize, and deploy AI models without managing infrastructure.


--------------------------------------------------------------------------------



In [29]:
# Example of streaming with Converse API
def stream_converse(model_id, messages, inference_config=None):
    if inference_config is None:
        inference_config = {}
    
    print("Streaming response (chunks will appear as they are received):\n")
    print("-" * 80)
    
    full_response = ""
    
    try:
        response = bedrock.converse_stream(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config
        )
        response_stream = response.get('stream')
        if response_stream:
            for event in response_stream:

                if 'messageStart' in event:
                    print(f"\nRole: {event['messageStart']['role']}")

                if 'contentBlockDelta' in event:
                    print(event['contentBlockDelta']['delta']['text'], end="")

                if 'messageStop' in event:
                    print(f"\nStop reason: {event['messageStop']['stopReason']}")

                if 'metadata' in event:
                    metadata = event['metadata']
                    if 'usage' in metadata:
                        print("\nToken usage")
                        print(f"Input tokens: {metadata['usage']['inputTokens']}")
                        print(
                            f":Output tokens: {metadata['usage']['outputTokens']}")
                        print(f":Total tokens: {metadata['usage']['totalTokens']}")
                    if 'metrics' in event['metadata']:
                        print(
                            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")

                
            print("\n" + "-" * 80)
        return full_response
    
    except Exception as e:
        print(f"Error in streaming: {str(e)}")
        return None

In [30]:
# Let's try streaming a longer summary
streaming_request = [
    {
        "role": "user",
        "content": [
            {
                "text": f"""Please provide a detailed summary of the following text, explaining its key points and implications:
                
                {text_to_summarize}
                
                Make your summary comprehensive but clear.
                """
            }
        ]
    }
]

In [31]:
# Only run this when you're ready to see streaming output
streamed_response = stream_converse(
    MODELS["Claude 3.7 Sonnet"], 
    streaming_request, 
    inference_config={"temperature": 0.4, "maxTokens": 1000}
)


Streaming response (chunks will appear as they are received):

--------------------------------------------------------------------------------

Role: assistant
# Summary: Amazon Bedrock - Democratizing Access to Foundation Models

## Key Points

Amazon has launched Amazon Bedrock, a new service that provides API access to Foundation Models (FMs) from multiple AI companies including:
- AI21 Labs
- Anthropic
- Stability AI
- Amazon's own models

The service is designed as a response to customer feedback, aiming to make generative AI more accessible to all developers.

## Core Features

1. **Diverse Model Access**: Offers a range of powerful FMs for both text and image generation

2. **Amazon Titan Models**: Introduces two new Large Language Models (LLMs) developed by Amazon

3. **Serverless Architecture**: Eliminates infrastructure management needs for users

4. **Customization Capabilities**: Allows users to privately fine-tune models with their own data

5. **AWS Integration**: Seamle

In [32]:
code_generation_prompt = """
Create a Python function called get_weather that accepts a location as parameter. \
The function should return a dictionary containing weather data (condition, temperature, and humidity) for predefined cities.\
Use a mock data structure instead of actual API calls. Include New York, San Francisco, Miami, and Seattle as default cities.\
The return statement should look like the following: return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0}).
Only return the function and no preamble or examples.
"""

In [33]:
converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": f"{code_generation_prompt}"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,
        "topP": 0.9,
        "maxTokens": 500
    }
}

In [34]:
try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )
    
    # Extract the model's response
    claude_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(claude_converse_response, "Claude 3.7 Sonnet (Converse API)")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from Claude 3.7 Sonnet (Converse API)

```python
def get_weather(location):
    weather_data = {
        "New York": {"condition": "Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 62, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 58, "humidity": 90}
    }
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})
```


--------------------------------------------------------------------------------



In [35]:
def get_weather(location):
    weather_data = {
        "New York": {"condition": "Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 62, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 58, "humidity": 90}
    }
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})

In [37]:
weather_tool = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_weather",
                "description": "Get current weather for a specific location",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city name to get weather for"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}  # Let the model decide when to use the tool
    }
}

In [38]:
function_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "What's the weather like in San Francisco right now? And what should I wear?"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,  # Use 0 temperature for deterministic function calling
        "maxTokens": 500
    }
}

In [39]:
response = bedrock.converse(
    modelId=MODELS["Claude 3.7 Sonnet"],
    messages=function_request["messages"],
    inferenceConfig=function_request["inferenceConfig"],
    toolConfig=weather_tool
)
print(json.dumps(response, indent=2))

{
  "ResponseMetadata": {
    "RequestId": "69ff54fb-7848-443d-a4f3-c6ee99468ada",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 27 May 2025 08:02:03 GMT",
      "content-type": "application/json",
      "content-length": "531",
      "connection": "keep-alive",
      "x-amzn-requestid": "69ff54fb-7848-443d-a4f3-c6ee99468ada"
    },
    "RetryAttempts": 1
  },
  "output": {
    "message": {
      "role": "assistant",
      "content": [
        {
          "text": "I can check the current weather in San Francisco for you and provide clothing recommendations based on that information."
        },
        {
          "toolUse": {
            "toolUseId": "tooluse_TQ6SJEM0RkeCqgd8dVLS0A",
            "name": "get_weather",
            "input": {
              "location": "San Francisco"
            }
          }
        }
      ]
    }
  },
  "stopReason": "tool_use",
  "usage": {
    "inputTokens": 404,
    "outputTokens": 75,
    "totalTokens": 479
  },
  "metrics":

In [40]:
def handle_function_calling(model_id, request, tool_config):
    try:
        # Step 1: Send initial request
        response = bedrock.converse(
            modelId=model_id,
            messages=request["messages"],
            inferenceConfig=request["inferenceConfig"],
            toolConfig=tool_config
        )
        
        # Check if the model wants to use a tool (check the correct response structure)
        content_blocks = response["output"]["message"]["content"]
        has_tool_use = any("toolUse" in block for block in content_blocks)
        
        if has_tool_use:
            # Find the toolUse block
            tool_use_block = next(block for block in content_blocks if "toolUse" in block)
            tool_use = tool_use_block["toolUse"]
            tool_name = tool_use["name"]
            tool_input = tool_use["input"]
            tool_use_id = tool_use["toolUseId"]
            
            # Step 2: Execute the tool
            if tool_name == "get_weather":
                tool_result = get_weather(tool_input["location"])
            else:
                tool_result = {"error": f"Unknown tool: {tool_name}"}
            
            # Step 3: Send the tool result back to the model
            updated_messages = request["messages"] + [
                {
                    "role": "assistant",
                    "content": [
                        {
                            "toolUse": {
                                "toolUseId": tool_use_id,
                                "name": tool_name,
                                "input": tool_input
                            }
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [
                                    {
                                        "json": tool_result
                                    }
                                ],
                                "status": "success"
                            }
                        }
                    ]
                }
            ]
            
            # Step 4: Get final response
            final_response = bedrock.converse(
                modelId=model_id,
                messages=updated_messages,
                inferenceConfig=request["inferenceConfig"],
                toolConfig=tool_config  
            )
            
            # Extract text from the correct response structure
            final_text = ""
            for block in final_response["output"]["message"]["content"]:
                if "text" in block:
                    final_text = block["text"]
                    break
            
            return {
                "tool_call": {"name": tool_name, "input": tool_input},
                "tool_result": tool_result,
                "final_response": final_text
            }
        else:
            # Model didn't use a tool, just return the text response
            text_response = ""
            for block in content_blocks:
                if "text" in block:
                    text_response = block["text"]
                    break
                    
            return {
                "final_response": text_response
            }
    
    except Exception as e:
        print(f"Error in function calling: {str(e)}")
        return {"error": str(e)}

In [41]:
function_result = handle_function_calling(
    MODELS["Claude 3.7 Sonnet"], 
    function_request,
    weather_tool
)

# Display the results
if "error" not in function_result:
    if "tool_call" in function_result:
        print(f"Tool Call: {function_result['tool_call']['name']}({function_result['tool_call']['input']})")
        print(f"Tool Result: {function_result['tool_result']}")
    
    display_response(function_result["final_response"], "Claude 3.7 Sonnet (Function Calling)")
else:
    print(f"Error: {function_result['error']}")

Tool Call: get_weather({'location': 'San Francisco'})
Tool Result: {'condition': 'Foggy', 'temperature': 62, 'humidity': 80}


### Response from Claude 3.7 Sonnet (Function Calling)

Currently in San Francisco, it's 62°F with foggy conditions and 80% humidity.

For this weather, I'd recommend:
- A light jacket or sweater as your outer layer
- Long pants or jeans
- A t-shirt or light long-sleeve shirt underneath
- Comfortable shoes

The foggy conditions mean it might feel a bit cooler than the temperature suggests, especially if you'll be near the water. Layers are always a good idea in San Francisco as the weather can change throughout the day. You might want to bring a light scarf or hat if you're sensitive to the damp, cool air that comes with fog.


--------------------------------------------------------------------------------

